<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/develop/examples/annotation_import/image.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>




<td>
<a href="https://github.com/Labelbox/labelbox-python/tree/develop/examples/annotation_import/image.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# Image Annotation Import
This notebook will provide examples of each supported annotation type for image assets. 

### [Model-assisted labeling (MAL)](https://docs.labelbox.com/v4/docs/model-assisted-labeling)
* This workflow allows you to import computer-generated predictions (or simply annotations created outside of Labelbox) as pre-labels on an asset. 
The imported annotations will be pre-populated in the labeling editor. However, in order to convert the pre-labels to real annotations, a human labeler will still need to open the Data Row in the Editor and submit it. This functionality is designed to speed up human labeling.


### [Import ground truth](https://docs.labelbox.com/v4/docs/import-ground-truth)
* This  workflow functionality allows you to bulk import your ground truth annotations from an external or third-party labeling system into Labelbox Annotate. Using the label import API to import external data is a useful way to consolidate and migrate all annotations into Labelbox as a single source of truth.


### Python Annotation types vs NDJSON
**Python Annotation Type (recommended)**
- Provides a seamless transition between third-party platforms, machine learning pipelines, and Labelbox.

- Allows you to build annotations locally with local file paths, numpy arrays, or URLs

- Easily convert Python Annotation Type format to NDJSON format to quickly import annotations to Labelbox

- It supports one-level nested classification (free text / radio / checklist) under the object or classification annotation.

**NDJSON**
- Skip formatting annotation payload in the Python Annotation Types format just to convert back to NDJSON

- Ability to create the payload in the NDJSON import format directly

- It supports any levels of nested classification (free text / radio / checklist) under the object or classification annotation.

## Imports

In [30]:
!pip install -q 'labelbox[data]'

In [31]:
from labelbox.schema.ontology import OntologyBuilder, Tool, Classification, Option
from labelbox import Client, MALPredictionImport, LabelImport
from labelbox.data.annotation_types import (
    Label, ImageData, ObjectAnnotation, MaskData,
    Rectangle, Point, Line, Mask, Polygon,
    Radio, Checklist, Text,
    ClassificationAnnotation, ClassificationAnswer
)
from labelbox.data.serialization import NDJsonConverter
from labelbox.schema.media_type import MediaType
import uuid
import numpy as np
from labelbox.schema.queue_mode import QueueMode


## Replace with your API KEY

Guides on [Create an API key](https://docs.labelbox.com/docs/create-an-api-key)

In [32]:
API_KEY = None
client = Client(API_KEY)

## Supported Annotations for Image


### Supported Annotation Types

In [33]:
########### Radio Classification ###########

# Python annotation
radio_annotation = ClassificationAnnotation(
    name="radio_question", 
    value=Radio(answer = ClassificationAnswer(name = "second_radio_answer"))
)

# NDJSON
radio_annotation_ndjson = {
  'name': 'radio_question',
  'answer': {'name': 'second_radio_answer'}
} 

In [34]:
########## Nested Radio Classification is only supported with NDJSON tools##########

# NDJSON 
nested_radio_annotation_ndjson = {
  "name": "nested_radio_question",
  "answer": {"name": "first_radio_answer"},
  "classifications" : [
   {'name': 'sub_radio_question', 'answer': {'name': 'first_sub_radio_answer'}}
   ]
}


In [35]:
############ Checklist question ############

# Python Annotations
checklist_annotation = ClassificationAnnotation(
  name="checklist_question", # must match your ontology feature's name
  value=Checklist(answer = [ClassificationAnswer(name = "first_checklist_answer"), ClassificationAnswer(name = "second_checklist_answer")])
 )

# NDJSON
checklist_annotation_ndjson = {
  'name': 'checklist_question',
  'answer': [
    {'name': 'first_checklist_answer'},
    {'name': 'second_checklist_answer'}
  ]
}

In [36]:
############# Free text Classification #############

# Python annotation
text_annotation = ClassificationAnnotation(
  name="free_text",  # must match your ontology feature's name
  value=Text(answer="sample text")
)


# NDJSON
text_annotation_ndjson = {
  'name': 'free_text',
  'answer': 'sample text',
}

In [37]:
####### Bounding box #######


# Python Annotation 
bbox_annotation = ObjectAnnotation(
  name = "bounding_box",  # must match your ontology feature's name
  value = Rectangle(
        start=Point(x=977, y=1690), # Top left
        end=Point(x=330, y=225), # Bottom right
    ),
)

#NDJSON 
bbox_annotation_ndjson = {
  'name': 'bounding_box',
  'bbox': {
          "top": 977,
          "left": 1690,
          "height": 330,
          "width": 225
      }
}


In [38]:
# Bounding box with nested classification
bbox_with_radio_subclass_annotation = ObjectAnnotation(
    name="bbox_with_radio_subclass", # must match your ontology feature's name
    value=Rectangle(
        start=Point(x=933, y=541), # Top left
        end=Point(x=191, y=330), # Bottom right
    ),
    classifications=[
    	ClassificationAnnotation(
        	name="sub_radio_question",
      		value=Radio(answer=ClassificationAnswer(name="first_sub_radio_answer"))
    )
  ]
)


## NDJSON
bbox_with_radio_subclass_ndjson = {
    "name": "bbox_with_radio_subclass", 
    "classifications": [{
        "name": "sub_radio_question",
        "answer": 
            { "name":"first_sub_radio_answer" }
         
    }],
    "bbox": {
          "top": 933,
          "left": 541,
          "height": 191,
          "width": 330
        }
}

In [39]:
########## Polygon ##########
# Python Anotation 
polygon_annotation = ObjectAnnotation(
  name = "polygon", # must match your ontology feature's name
  value=Polygon( # Coordinates for the verticies of your polygon
        points=[Point(x=1489.581,y=183.934),Point(x=2278.306,y=256.885),Point(x=2428.197,y=200.437),Point(x=2560.0,y=335.419),
                Point(x=2557.386,y=503.165),Point(x=2320.596,y=503.103),Point(x=2156.083, y=628.943),Point(x=2161.111,y=785.519),
                Point(x=2002.115, y=894.647),Point(x=1838.456,y=877.874),Point(x=1436.53,y=874.636),Point(x=1411.403,y=758.579),
                Point(x=1353.853,y=751.74),Point(x=1345.264, y=453.461),Point(x=1426.011,y=421.129)]
    ),
)


# NDJSON

polygon_annotation_ndjson = {
  'name': 'polygon',
  'polygon': [
    {'x': 1489.581, 'y': 183.934},
    {'x': 2278.306, 'y': 256.885},
    {'x': 2428.197, 'y': 200.437},
    {'x': 2560.0, 'y': 335.419},
    {'x': 2557.386, 'y': 503.165},
    {'x': 2320.596, 'y': 503.103},
    {'x': 2156.083, 'y': 628.943},
    {'x': 2161.111, 'y': 785.519},
    {'x': 2002.115, 'y': 894.647},
    {'x': 1838.456, 'y': 877.874},
    {'x': 1436.53, 'y': 874.636},
    {'x': 1411.403, 'y': 758.579},
    {'x': 1353.853, 'y': 751.74},
    {'x': 1345.264, 'y': 453.461},
    {'x': 1426.011, 'y': 421.129},
    {'x': 1489.581, 'y': 183.934}
  ]
}

In [40]:
######### Mask #########

# Python 
# Identifying what values in the numpy array correspond to the mask annotation
color = (0, 0, 0)

# convert a polygon to mask
im_height, im_width = 100,100 #need to provide the height and width of image.
mask_data = MaskData(arr=
                     polygon_annotation.value.draw(height=im_height,width=im_width,color=color))

# convert a 2D array to 3D array
arr_2d = np.zeros((100,100), dtype='uint8')
mask_data = MaskData.from_2D_arr(arr_2d)

# a 3D array where 3rd axis is RGB values.
mask_data = MaskData(arr= np.zeros([400,450,3],dtype='uint8'))

mask_annotation = ObjectAnnotation(
  name = "mask", # must match your ontology feature's name
  value = Mask(mask=mask_data, color=color),
)


# NDJSON
mask_annotation_ndjson = {
  'name': 'mask',
  'classifications': [],
  'mask': {'instanceURI': 'https://storage.labelbox.com/cjhfn5y6s0pk507024nz1ocys%2F1d60856c-59b7-3060-2754-83f7e93e0d01-1?Expires=1666901963361&KeyName=labelbox-assets-key-3&Signature=t-2s2DB4YjFuWEFak0wxYqfBfZA',
  'colorRGB': (0, 0, 0)}
}



In [41]:
######## Point Annotation ########

# Python Annotation
point_annotation = ObjectAnnotation(
  name = "point",  # must match your ontology feature's name
  value = Point(x=1166.606, y=1441.768),
)


# NDJSON
point_annotation_ndjson = {
  'name': 'point',
  'classifications': [],
  'point': {'x': 1166.606, 'y': 1441.768}
}

In [42]:
###### Polygon ######


# Python Annotation 

polyline_annotation = ObjectAnnotation(
  name = "polyline", # must match your ontology feature's name
  value=Line( # Coordinates for the keypoints in your polyline
        points=[Point(x=2534.353, y=249.471),Point(x=2429.492, y=182.092),Point(x=2294.322, y=221.962),Point(x=2224.491, y=180.463),Point(x=2136.123, y=204.716),
                Point(x=1712.247, y=173.949),Point(x=1703.838, y=84.438),Point(x=1579.772, y=82.61),Point(x=1583.442, y=167.552),
                Point(x=1478.869, y=164.903),Point(x=1418.941, y=318.149),Point(x=1243.128, y=400.815),Point(x=1022.067, y=319.007),
                Point(x=892.367, y=379.216),Point(x=670.273, y=364.408),Point(x=613.114, y=288.16),Point(x=377.559, y=238.251),
                Point(x=368.087, y=185.064),Point(x=246.557, y=167.286),Point(x=236.648, y=285.61),Point(x=90.929, y=326.412)]
    ),
)

# NDJSON
polyline_annotation_ndjson = {
  'name': 'polyline',
  'classifications': [],
  'line': [
    {'x': 2534.353, 'y': 249.471},
    {'x': 2429.492, 'y': 182.092},
    {'x': 2294.322, 'y': 221.962},
    {'x': 2224.491, 'y': 180.463},
    {'x': 2136.123, 'y': 204.716},
    {'x': 1712.247, 'y': 173.949},
    {'x': 1703.838, 'y': 84.438},
    {'x': 1579.772, 'y': 82.61},
    {'x': 1583.442, 'y': 167.552},
    {'x': 1478.869, 'y': 164.903},
    {'x': 1418.941, 'y': 318.149},
    {'x': 1243.128, 'y': 400.815},
    {'x': 1022.067, 'y': 319.007},
    {'x': 892.367, 'y': 379.216},
    {'x': 670.273, 'y': 364.408},
    {'x': 613.114, 'y': 288.16},
    {'x': 377.559, 'y': 238.251},
    {'x': 368.087, 'y': 185.064},
    {'x': 246.557, 'y': 167.286},
    {'x': 236.648, 'y': 285.61},
    {'x': 90.929, 'y': 326.412}
  ]
}


# Upload Annotations - putting it all together


## Step 1: Import data rows into Catalog



In [45]:
# send a sample image as batch to the project


test_img_url = {
    "row_data": "https://storage.googleapis.com/labelbox-datasets/image_sample_data/2560px-Kitano_Street_Kobe01s5s4110.jpeg",
    "global_key": str(uuid.uuid4())
}
  

dataset = client.create_dataset(name="demo_dataset_img")
data_row = dataset.create_data_row(test_img_url)
print(data_row)

len(test_img_url)

<DataRow {
    "created_at": "2023-01-30 17:43:42+00:00",
    "external_id": null,
    "global_key": "3ed74b59-074c-4cb6-98bf-b4c3098ab2a6",
    "media_attributes": {},
    "metadata": [],
    "metadata_fields": [],
    "row_data": "https://raw.githubusercontent.com/Labelbox/labelbox-python/develop/examples/assets/2560px-Kitano_Street_Kobe01s5s4110.jpg",
    "uid": "cldj3nso96t5l07xd9d1nf46s",
    "updated_at": "2023-01-30 17:43:42+00:00"
}>


2

## Step 2: Create/select an Ontology
Your project should have the correct ontology setup with all the tools and classifications supported for your annotations, and the tool names and classification instructions should match the `name`/`instructions` fields in your annotations to ensure the correct feature schemas are matched.

For example, when we create the bounding box annotation above, we provided the `name` as `polyline`. Now, when we setup our ontology, we must ensure that the name of my bounding box tool is also `polyline`. The same alignment must hold true for the other tools and classifications we create in our ontology.




In [46]:

ontology_builder = OntologyBuilder(
  classifications=[ # List of Classification objects
    Classification( # Radio classification given the name "text" with two options: "first_radio_answer" and "second_radio_answer"
      class_type=Classification.Type.RADIO, 
      instructions="radio_question", 
      options=[
        Option(value="first_radio_answer"),
        Option(value="second_radio_answer")
      ]
    ),
    Classification( # Checklist classification given the name "text" with two options: "first_checklist_answer" and "second_checklist_answer"
      class_type=Classification.Type.CHECKLIST, 
      instructions="checklist_question", 
      options=[
        Option(value="first_checklist_answer"),
        Option(value="second_checklist_answer")            
      ]
    ), 
     Classification( # Text classification given the name "text"
      class_type=Classification.Type.TEXT,
      instructions="free_text"
      ),
      Classification(
          class_type=Classification.Type.RADIO, 
          instructions="nested_radio_question",
          options=[
              Option("first_radio_answer",
                  options=[
                      Classification(
                          class_type=Classification.Type.RADIO,
                          instructions="sub_radio_question",
                          options=[Option("first_sub_radio_answer")]
                      )
                  ]
              )
            ] 
          )    
    
    ],
  tools=[ # List of Tool objects
    Tool( # Bounding Box tool given the name "box"
      tool=Tool.Type.BBOX, 
      name="bounding_box"), 
    Tool( # Bounding Box tool given the name "box"
      tool=Tool.Type.BBOX, 
      name="bbox_with_radio_subclass",
      classifications=[
            Classification(
                class_type=Classification.Type.RADIO,
                instructions="sub_radio_question",
                options=[
                  Option(value="first_sub_radio_answer")
                ]
              ),
        ]
      ), 
    Tool( # Polygon tool given the name "polygon"
      tool=Tool.Type.POLYGON, 
      name="polygon"),
    Tool( # Segmentation mask tool given the name "mask"
      tool=Tool.Type.SEGMENTATION, 
      name="mask"),
 	  Tool( # Point tool given the name "point"
      tool=Tool.Type.POINT, 
      name="point"), 
    Tool( # Polyline tool given the name "line"
      tool=Tool.Type.LINE, 
      name="polyline"),]
)

ontology = client.create_ontology("Ontology Image Annotations", ontology_builder.asdict())


## Step 3: Create a labeling project

Connect the ontology to the labeling project

In [47]:

# create a project and configure the ontology 
project = client.create_project(
    name="annotations_import_project_demo",
    media_type=MediaType.Image,
    queue_mode=QueueMode.Batch)

project.setup_editor(ontology) # Connect your ontology and editor to your MAL project

## Step 4: Send a batch of data rows to the project


In [48]:
batch = project.create_batch(
    "Initial batch2", # name of the batch
    dataset.export_data_rows(), # list of Data Rows
  1 # priority between 1-5
)
print("Batch", batch)

Batch <Batch {
    "consensus_settings_json": "{\"numberOfLabels\":1,\"coveragePercentage\":0}",
    "created_at": "2023-01-30 17:43:58+00:00",
    "name": "Initial batch2",
    "size": 1,
    "uid": "acef2a70-a0c5-11ed-ac19-71d552e4dbe5",
    "updated_at": "2023-01-30 17:43:58+00:00"
}>


## Step 5: Create the annotations payload

Create the annotations payload using the snippets of code above

Labelbox support two formats for the annotations payload: NDJSON and Python Annotation types. Both are described below. If you are using Python Annotation types, compose your annotations into Labels attached to the data rows.

### Python Annotations

Here we create the complete label ndjson payload of annotations only using python annotation format. There is one annotation for each reference to an annotation that we created.

In [49]:
# create a Label

label = []
for data_row in dataset.export_data_rows():
  label.append(Label(
      data=ImageData(
          uid=data_row.uid),
      annotations = [
          checklist_annotation, 
          text_annotation,
          bbox_annotation, 
          bbox_with_radio_subclass_annotation, 
          polygon_annotation, 
          mask_annotation, 
          point_annotation, 
          polyline_annotation
      ]
  )
)


# Convert our label from a Labelbox class object to the underlying NDJSON format required for upload 
label_ndjson = list(NDJsonConverter.serialize(label))

### NDJSON annotations
Here we create the complete label ndjson payload of annotations only using NDJSON format. There is one annotation for each reference to an annotation that we created above.

In [50]:
label_ndjson_method2 = []
for annotation in [radio_annotation_ndjson, 
              checklist_annotation_ndjson, 
              text_annotation_ndjson,
              bbox_annotation_ndjson, 
              bbox_with_radio_subclass_ndjson, 
              polygon_annotation_ndjson, 
              mask_annotation_ndjson, 
              point_annotation_ndjson, 
              polyline_annotation_ndjson,
              nested_radio_annotation_ndjson]:
  annotation.update({
      'uuid': str(uuid.uuid4()),
      'dataRow': {'id': data_row.uid},
  })
  label_ndjson_method2.append(annotation)


## Step 6: Upload annotations to a project as pre-labels or complete labels

### Model-Assisted Labeling
For the purpose of this tutorial only run one of the label_ndjosn  annotation type tools at the time (NDJSON or Annotation types). Delete the previous labels before uploading labels that use the 2nd method (ndjson)


In [51]:
# Upload MAL label for this data row in project
upload_job = MALPredictionImport.create_from_objects(
    client = client, 
    project_id = project.uid, 
    name="mal_job"+str(uuid.uuid4()), 
    predictions=label_ndjson_method2)

print("Errors:", upload_job.errors)
print(" ")

Errors: []
 


### Label Import


In [54]:
# Upload label for this data row in project
upload_job = LabelImport.create_from_objects(
    client = client, 
    project_id = project.uid, 
    name="label_import_job"+str(uuid.uuid4()),  
    labels=label_ndjson_method2)

print("Errors:", upload_job.errors)
print(" ")

Errors: []
 


In [ ]:
# project.delete()
# dataset.delete()